In [1]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

import seaborn as sns           

import numpy as np

import scipy.stats as stats
import math

pd.set_option('display.max_rows',1600)
pd.set_option('display.max_columns',1600)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor,AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score

from sklearn.metrics import r2_score, mean_squared_error, make_scorer


In [2]:
import xlsxwriter

import pandas as pd

from pandas import DataFrame

path = (r"C:\Users\Raviteja\Desktop\Python_Practice\FLiPR_Hackathon\Train_dataset.xlsx")

xl = pd.ExcelFile(path)

print(xl.sheet_names)

FLiPR_df = xl.parse('Train_dataset')

['Train_dataset', 'Diuresis_TS']


In [3]:
path = (r"C:\Users\Raviteja\Desktop\Python_Practice\FLiPR_Hackathon\Test_dataset.xlsx")

xl = pd.ExcelFile(path)

print(xl.sheet_names)

Test_FLiPR_df = xl.parse('Test_dataset')


['Test_dataset']


In [4]:
print(FLiPR_df.dtypes)

FLiPR_df.shape

print(FLiPR_df.columns)

FLiPR_df.isnull().sum()

people_ID = FLiPR_df['people_ID']

y = FLiPR_df['Infect_Prob']

z= Test_FLiPR_df['people_ID']

drop_Columns=['people_ID','Infect_Prob']


Output_1 = pd.DataFrame({'people_ID':z})

people_ID                   int64
Region                     object
Gender                     object
Designation                object
Name                       object
Married                    object
Children                  float64
Occupation                 object
Mode_transport             object
cases/1M                    int64
Deaths/1M                   int64
comorbidity                object
Age                         int64
Coma score                  int64
Pulmonary score            object
cardiological pressure     object
Diuresis                  float64
Platelets                 float64
HBB                       float64
d-dimer                   float64
Heart rate                float64
HDL cholesterol           float64
Charlson Index              int64
Blood Glucose               int64
Insurance                 float64
salary                      int64
FT/month                  float64
Infect_Prob               float64
dtype: object
Index(['people_ID', 'Region', 'Gen

In [5]:
FLiPR_df.drop(columns=drop_Columns,inplace=True)
Test_FLiPR_df.drop(columns='people_ID',inplace=True)

In [6]:
missing_value_df=pd.DataFrame({'Column_Name':FLiPR_df.columns,
                             'missing_value_count':FLiPR_df.isnull().sum(),
                             'missing_value_percent':FLiPR_df.isnull().sum()/FLiPR_df.shape[0]*100})

missing_value_df.sort_values(by='missing_value_percent',ascending=False)

cols_drop=list(missing_value_df[missing_value_df['missing_value_percent']>50]['Column_Name'])

cols_drop

[]

In [7]:
for col in FLiPR_df.columns:
    if FLiPR_df[col].nunique()==FLiPR_df.shape[0] or FLiPR_df[col].nunique()==1:
        cols_drop.append(col)
cols_drop


[]

In [8]:
FLiPR_df.drop(columns=cols_drop,inplace=True)
Test_FLiPR_df.drop(columns=cols_drop,inplace=True)
FLiPR_df.shape

(10714, 26)

In [9]:
cols_change=['Children','Age']
for col in cols_change:
        FLiPR_df[col]=FLiPR_df[col].astype(object)
        Test_FLiPR_df[col]=Test_FLiPR_df[col].astype(object)
FLiPR_df.dtypes

Region                     object
Gender                     object
Designation                object
Name                       object
Married                    object
Children                   object
Occupation                 object
Mode_transport             object
cases/1M                    int64
Deaths/1M                   int64
comorbidity                object
Age                        object
Coma score                  int64
Pulmonary score            object
cardiological pressure     object
Diuresis                  float64
Platelets                 float64
HBB                       float64
d-dimer                   float64
Heart rate                float64
HDL cholesterol           float64
Charlson Index              int64
Blood Glucose               int64
Insurance                 float64
salary                      int64
FT/month                  float64
dtype: object

In [10]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(FLiPR_df,y,test_size=0.3,random_state=43)

X_train.shape

(7499, 26)

In [11]:
for col in X_train.columns:
    if X_train[col].dtype=='object':
        X_train[col]=X_train[col].fillna(X_train[col].mode()[0]).astype(object)
        X_test[col]=X_test[col].fillna(X_train[col].mode()[0]).astype(object)
        Test_FLiPR_df[col]=Test_FLiPR_df[col].fillna(X_train[col].mode()[0]).astype(object)
    
    else:
        X_train[col]=X_train[col].fillna(X_train[col].mean())
        X_test[col]=X_test[col].fillna(X_train[col].mean())
        Test_FLiPR_df[col]=Test_FLiPR_df[col].fillna(X_train[col].mean())
        
X_train.shape

(7499, 26)

In [12]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import numpy as np


class LabelEncoderExt(object):
    def __init__(self):
        """
        It differs from LabelEncoder by handling new classes and providing a value for it [Unknown]
        Unknown will be added in fit and transform will take care of new item. It gives unknown class id
        """
        self.label_encoder = LabelEncoder()
        # self.classes_ = self.label_encoder.classes_

    def fit(self, data_list):
        """
        This will fit the encoder for all the unique values and introduce unknown value
        :param data_list: A list of string
        :return: self
        """
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_

        return self

    def transform(self, data_list):
        """
        This will transform the data_list to id list where the new values get assigned to Unknown class
        :param data_list:
        :return:
        """
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]

        return self.label_encoder.transform(new_data_list)
    
le=LabelEncoderExt()
mm=MinMaxScaler()

for col in X_train.columns:
    if X_train[col].dtype=='object':
        le.fit(X_train[col])
        X_train[col]=le.transform(X_train[col])
        X_test[col]=le.transform(X_test[col])
        Test_FLiPR_df[col]=le.transform(Test_FLiPR_df[col])
    else:
        mm.fit([X_train[col]])
        X_train[col]=mm.fit_transform(np.array(X_train[col]).reshape(-1,1))
        X_test[col]=mm.transform(np.array(X_test[col]).reshape(-1,1))
        Test_FLiPR_df[col]=mm.transform(np.array(Test_FLiPR_df[col]).reshape(-1,1))
        


In [13]:
y_train.isnull().sum()

0

In [14]:
from sklearn.linear_model import LinearRegression

ligReg = LinearRegression()

ligReg.fit(X_train, y_train)

y_predict = ligReg.predict(X_test)



In [15]:
from sklearn.metrics import r2_score, mean_squared_error

print(r2_score(y_test, y_predict))

print(mean_squared_error(y_test, y_predict))

print(ligReg.score(X_train, y_train))

print(ligReg.score(X_test,y_test))

print(Output_1.shape)

print(Test_FLiPR_df.columns)

print(X_test.columns)

0.24665841833444868
85.5340783594381
0.2601467172435087
0.24665841833444868
(14498, 1)
Index(['Region', 'Gender', 'Designation', 'Name', 'Married', 'Children',
       'Occupation', 'Mode_transport', 'cases/1M', 'Deaths/1M', 'comorbidity',
       'Age', 'Coma score', 'Pulmonary score', 'cardiological pressure',
       'Diuresis', 'Platelets', 'HBB', 'd-dimer', 'Heart rate',
       'HDL cholesterol', 'Charlson Index', 'Blood Glucose', 'Insurance',
       'salary', 'FT/month'],
      dtype='object')
Index(['Region', 'Gender', 'Designation', 'Name', 'Married', 'Children',
       'Occupation', 'Mode_transport', 'cases/1M', 'Deaths/1M', 'comorbidity',
       'Age', 'Coma score', 'Pulmonary score', 'cardiological pressure',
       'Diuresis', 'Platelets', 'HBB', 'd-dimer', 'Heart rate',
       'HDL cholesterol', 'Charlson Index', 'Blood Glucose', 'Insurance',
       'salary', 'FT/month'],
      dtype='object')


In [16]:
models = {
           'LR': LinearRegression(), 
          'DTR': DecisionTreeRegressor(), 
          'RFS': RandomForestRegressor(),
          'GBR': GradientBoostingRegressor(),
          'ADB': AdaBoostRegressor(),
         }

for mName, model in models.items():
    obj = models[mName]
    obj.fit(X_train, y_train)
    y_predict = obj.predict(X_test)
    
    obj_predict = obj.predict(Test_FLiPR_df)
    
    train_acc = obj.score(X_train, y_train)
    test_acc =  obj.score(X_test, y_test)
    
    R2Square = r2_score(y_test, y_predict)
    MSquareE = mean_squared_error(y_test, y_predict)
    
    Output_1[mName] = obj_predict
    
    print("\n\n")
    print("{} Model TrainAcc: ====>{} TestAccuracy          : ====>{}".format(mName, train_acc, test_acc))
    print("{} Model R2 Score: ====>{} Model MeanSquare Error: ====>{}".format(mName, R2Square, MSquareE))




LR Model TrainAcc: ====>0.2601467172435087 TestAccuracy          : ====>0.24665841833444868
LR Model R2 Score: ====>0.24665841833444868 Model MeanSquare Error: ====>85.5340783594381



DTR Model TrainAcc: ====>0.999999999999806 TestAccuracy          : ====>-0.5039915705546609
DTR Model R2 Score: ====>-0.5039915705546609 Model MeanSquare Error: ====>170.7625544355894



RFS Model TrainAcc: ====>0.8959333288289036 TestAccuracy          : ====>0.25694291824692095
RFS Model R2 Score: ====>0.25694291824692095 Model MeanSquare Error: ====>84.36638067380632



GBR Model TrainAcc: ====>0.37332780324546677 TestAccuracy          : ====>0.1886677269606997
GBR Model R2 Score: ====>0.18866772696069967 Model MeanSquare Error: ====>92.11831645381469



ADB Model TrainAcc: ====>0.2680844072444837 TestAccuracy          : ====>0.24238771918138236
ADB Model R2 Score: ====>0.24238771918138236 Model MeanSquare Error: ====>86.01897169985394


In [17]:
Output_1.to_csv('Output_1.csv')

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

svcdtree=SVR()

parameter_grid={'C':[1,5,10,15],
                'kernel':['linear','rbf'],
               'gamma':[0.001,0.01,0.015,0.1]
               }

grid=RandomizedSearchCV(svcdtree,cv=5,param_distributions=parameter_grid)

grid.fit(X_train, y_train)
  
grid.best_score_

best_model=grid.best_estimator_

best_model.fit(X_train, y_train)

y_predict = best_model.predict(X_test)

print("Train Accuracy(R2 Score): ===========>", best_model.score(X_train, y_train), "\n")
print("Test Accuracuy(R2 Score): ===========>", best_model.score(X_test, y_test), "\n")
